In [202]:
#%pip install openpyxl

In [203]:
import pandas as pd
import json
import os
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go

In [205]:
# Read Transaction Excel data into a Pandas dataframe
df = pd.read_excel('C:/Users/admin/Documents/GitHub/davemccallme.github.io/scratch/sunbursts graphs2/exceldata-trxn.xlsx')
df['Value'] = pd.to_numeric(df['Value'])
print(df.head())

  Root Level                 Level 1                          Level 2   Value
0        Web            Account Mgmt          Change Billing Address    83947
1        Web            Account Mgmt            Add/Edit Phone Number  137600
2        Web            Account Mgmt      Change User Name, Password   159607
3        Web  Alerts & Notifications  Change Billing & Payment Alerts  302380
4        Web  Alerts & Notifications                    Energy Alerts  233548


In [206]:
# Convert dataframe to a hierarchical dictionary
def make_hierarchy_dict(df, path=[], value_col="Value", threshold=10):
    if len(path) == df.columns.nlevels - 1:
        total = int(df[value_col].sum())
        if total < threshold:
            return None
        else:
            return total
    else:
        level = df.columns[len(path)]
        subdict = {}
        for key in df[level].dropna().unique():
            subdf = df[df[level] == key].drop(columns=[level])
            subhierarchy_dict = make_hierarchy_dict(subdf, path + [key], value_col, threshold)
            if subhierarchy_dict is not None:
                subdict[key] = subhierarchy_dict
        if len(subdict) == 0:
            return None
        else:
            return subdict

hierarchy_dict = make_hierarchy_dict(df.set_index(list(df.columns[:3])), value_col="Value", threshold=10)

print(hierarchy_dict)

127553926


In [207]:
# Convert hierarchical dictionary to Plotly-compatible JSON
def make_plotly_sunburst(hierarchy_dict, name=""):
    if isinstance(hierarchy_dict, dict):
        children = []
        for key, value in hierarchy_dict.items():
            children.append(make_plotly_sunburst(value, key))
        return {
            "name": name,
            "children": children
        }
    else:
        return {
            "label": name,
            "value": hierarchy_dict
        }

In [208]:
plotly_data = make_plotly_sunburst(hierarchy_dict)
print(plotly_data)

{'label': '', 'value': 127553926}


In [209]:
# Write JSON data to a file
with open("plotly_data.json", "w") as f:
    json.dump(plotly_data, f)

In [ ]:
#Error Check rows for data discrepencies
for i, row in df.iterrows():
    try:
        fig = px.sunburst(df.loc[[i]], path=["Root Level", "Level 1", "Level 2"], values='Value')
    except ValueError as e:
        print(e)

In [210]:
# Create a sunburst chart using Plotly Express
fig = px.sunburst(df, path=["Root Level", "Level 1", "Level 2"], values='Value')
fig.show()

In [211]:
# Format a sunburst chart using Plotly Express
color_sequence = ['darkblue', 'blue', 'lightblue']

fig = px.sunburst(df, path=["Root Level", "Level 1", "Level 2"], values='Value', 
                  color_discrete_sequence=color_sequence, hover_name='Value')

# Update the layout
fig.update_layout(
    title='Transaction Channel Volume 2022',
    font=dict(size=14),
    margin=dict(t=50, b=0, l=0, r=0),
    width=640,
    height=480,
    sunburstcolorway=color_sequence
)

In [212]:
# Format a sunburst chart using Plotly Express
color_sequence = ['darkorange', 'lightgrey', 'lightblue', 'tan']
fig = px.sunburst(df, path=["Root Level", "Level 1", "Level 2"], values='Value', 
                  hover_data=['Value'],
                  color_discrete_sequence=px.colors.qualitative.Dark2)

# format hover tooltip
fig.update_traces(hovertemplate='%{label}<br>%{value:,.0f}')
# Update the layout
fig.update_layout(
    title='Transaction Channel Volume 2022',
    font=dict(size=14),
    margin=dict(t=50, l=0, r=0, b=0),
    width=1400,
    height=900,
    sunburstcolorway=color_sequence,
)
fig.show()

In [ ]:
# Export the chart as an HTML file
pyo.plot(fig, filename='Trxn_sunburst_chart.html', auto_open=True)

'Trxn_sunburst_chart.html'